# Задание 9. Про ЕГЭ

В этом задании Вам предстоит выполнить роль аналитика данных в области образования.

Заказчик (руководство вуза) предоставил обезличенные данные с баллами ЕГЭ студентов (`stud_spisok_ege.csv`) и сведения о баллах за контрольные работы (`stud_spisok_ball.csv`).

Анализ этих данных позволит руководству вуза принимать обоснованные решения по повышению качества процесса обучения и выстраивать персональные образовательные маршруты (построить рекомендательную систему на основе искусственного интеллекта).

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/data_stud/stud_spisok_ege.csv"

df_1 = pd.read_csv(url, sep=';')
df_1.head()

,stud_kod,predmet_name,ball,year,svid_status
0,50095,Русский язык,76,2013,Действительно
1,50095,Английский язык,94,2013,Действительно
2,50095,Математика,56,2013,Действительно
3,50095,Обществознание,70,2013,Действительно
4,50104,Русский язык,72,2013,Действительно


- `stud_kod` - идентификатор студента
- `predmet_name` - имя предмета
- `ball` - балл
- `year` - год сдачи
- `svid_status` - статус свидетельства о сдаче ЕГЭ

In [2]:
url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/data_stud/stud_spisok_ball.csv"

df_2 = pd.read_csv(url, sep=';')
df_2.head()

,stud_kod,uch_plan_kod,predmet_name,ball,ts,cd_nomer,semestr,cd_type_name,uch_plan_punkt_kod
0,8,1,История,25,25-дек-11,1,1,Эссе,255
1,8,1,История,30,25-дек-11,2,1,Презентация,255
2,8,1,История,45,NaN,3,1,Аналитическая работа,255
3,8,1,Философия,15,30-апр-15,2,1,Реферат,293
4,8,1,Философия,18,30-апр-15,1,1,Аналитическая работа,293


- `stud_kod` - идентификатор студента
- `uch_plan_kod` - внутренний код учебного плана
- `predmet_name` - имя дисциплины по учебному плану
- `ball` - балл за контрольную работу
- `ts` - время, когда оценка внесена в систему
- `cd_nomer` - номер контрольной работы, если их несколько за семестр
- `semestr` - семестр
- `cd_type_name` - вид контрольной работы
- `uch_plan_punkt_kod` - идентификатор учебного плана

In [3]:
#df_2.dtypes

**Задача 1**: определите, всего сколько студентов представлено в наборе данных.

In [4]:
df_1['stud_kod'].nunique() + df_2['stud_kod'].nunique()

3286

**Задача 2**: определите по данным ЕГЭ те предметы, которые чаще всего сдают абитуренты в этом вузе.

In [5]:
df_1['predmet_name'].value_counts().idxmax()

'Математика'

**Задача 3**: определите средние, минимальные и максимальные баллы ЕГЭ по отдельным предметам.

In [6]:
subject_stats = df_1.groupby('predmet_name')['ball'].agg(['mean', 'min', 'max'])
subject_stats

,mean,min,max
predmet_name,,,
Английский язык,69.949640,0,100
Английский язык (устный),0.000000,0,0
Биология,55.557377,0,93
География,64.333333,37,85
Информатика и ИКТ,62.481928,0,94
История,59.945402,0,100
Литература,63.283333,0,87
Математика,64.135454,0,100
Немецкий язык,66.285714,23,90


**Задача 4**: определите годы с максимальными и минимальными показателями ЕГЭ.

In [7]:
yearly_stats = df_1.groupby('year')['ball'].mean()
max_year = yearly_stats.idxmax()
min_year = yearly_stats.idxmin()
max_year, min_year

(2013, 2009)

**Задача 5**: определите наиболее часто встречающийся вид контрольной работы в вузе.

In [8]:
df_2['cd_type_name'].value_counts().idxmax()

'Контрольная работа'

**Задача 6**: определите дисциплину(ы), вызвавшую наибольшие сложности у студентов.

Это позволит усилить подготовку по данным дисциплинам через внедрение факультативных занятий.

In [9]:
df_2['ball'] = df_2['ball'].str.replace(',', '.')
df_2['ball_numeric'] = pd.to_numeric(df_2['ball'], errors='coerce')
df_2.groupby('predmet_name')['ball_numeric'].mean().idxmin()

'Иностранный язык профессиональный'

**Задача 7**: определите отстающих студентов (претендентов на отчисление).

In [10]:
# Пусть претенденты на отчисление те, у кого меньше 50 баллов
average_scores_2 = df_2.groupby('stud_kod')['ball_numeric'].mean()
average_scores_2[average_scores_2 < 50]

stud_kod
8       21.322347
9       20.133600
21      18.537643
55      16.510554
70      18.049667
          ...    
2633    21.443063
2642    23.023891
2653    14.355080
2663    19.482314
2682    16.209921
Name: ball_numeric, Length: 285, dtype: float64

**Задача 8**: определите семестр, наиболее сложный для отдельных (выбранных) студентов.

In [11]:
average_scores_by_semester = df_2.groupby(['stud_kod', 'semestr'])['ball_numeric'].mean().reset_index()
average_scores_by_semester.loc[average_scores_by_semester.groupby('stud_kod')['ball_numeric'].idxmin()]

,stud_kod,semestr,ball_numeric
5,8,6,18.178571
12,9,5,17.612903
17,21,2,15.109730
29,55,6,13.907407
37,70,6,15.081481
...,...,...,...
2200,2633,2,18.794324
2208,2642,2,19.689459
2219,2653,5,4.400000
2225,2663,6,15.478571
